# **0. ToolBox**

In [1]:
!pip install thop
!pip install facenet-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 474.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 512.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [48]:
#
import cv2
import os
from facenet_pytorch import MTCNN

#
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.transforms import Resize
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, TensorDataset
#
import torchvision
from PIL import Image
#
import torch.nn as nn
import torch.nn.functional as F
#
from thop import profile
from thop import clever_format
#
import numpy as np
from skimage.feature import local_binary_pattern

# **1. Filepath**

In [2]:
# File Path
fp      = "/content/drive/MyDrive/Class/ML"
fp_ds   = fp+'/dataset'
fp_face = fp+'/face'
fp_body = fp+'/body'
fp_HOG  = fp+'/hog'
fp_LBP  = fp+'/lbp'

# **2. Preprocessing / Augmentation**

In [8]:
# Crop Face (MTCNN)
def crop_faces(folder_path, save_cropped=False, save_path=fp_face, subdir='/train'):
    input_directory = folder_path + subdir
    output_directory_faces = save_path + '2' + subdir
    output_directory_lbp = save_path + '3' + subdir

    # Create the output directories if they do not exist
    os.makedirs(output_directory_faces, exist_ok=True)
    os.makedirs(output_directory_lbp, exist_ok=True)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    mtcnn = MTCNN(keep_all=True, device=device)

    for filename in os.listdir(input_directory):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(input_directory, filename)
            image = Image.open(image_path)
            image_rgb = np.array(image)

            boxes, probs = mtcnn.detect(image)

            if boxes is not None:
                for i, (box, prob) in enumerate(zip(boxes, probs)):
                    if prob > 0.5:
                        x_min, y_min, x_max, y_max = map(int, box)
                        # Validate box dimensions to ensure it's not empty
                        if x_min >= x_max or y_min >= y_max:
                            print(f"Invalid box dimensions for {filename}: ({x_min}, {y_min}, {x_max}, {y_max})")
                            continue

                        if x_min < 0:
                            x_min = 0
                        if y_min < 0:
                            y_min = 0
                        if x_max > image_rgb.shape[1]:
                            x_max = image_rgb.shape[1]
                        if y_max > image_rgb.shape[0]:
                            y_max = image_rgb.shape[0]

                        face_image = image_rgb[y_min:y_max, x_min:x_max]
                        if face_image.size == 0:
                            print(f"No data in cropped face for {filename}.")
                            continue

                        gray_face = cv2.cvtColor(face_image, cv2.COLOR_RGB2GRAY)
                        equalized_face = cv2.equalizeHist(gray_face)

                        lbp = local_binary_pattern(equalized_face, 8, 1, method='uniform')
                        lbp_normalized = cv2.normalize(lbp, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

                        # Save cropped face and LBP images if requested
                        if save_cropped:
                            face_output_path = os.path.join(output_directory_faces, f'{filename[:-4]}_{i}.jpg')
                            lbp_output_path = os.path.join(output_directory_lbp, f'{filename[:-4]}_{i}.jpg')
                            Image.fromarray(face_image).save(face_output_path)
                            cv2.imwrite(lbp_output_path, lbp_normalized)

In [9]:
# Crop Face & Save
crop_faces(fp_ds, save_cropped=True, subdir='/train/adults')
crop_faces(fp_ds, save_cropped=True, subdir='/train/children')
crop_faces(fp_ds, save_cropped=True, subdir='/test/adults')
crop_faces(fp_ds, save_cropped=True, subdir='/test/children')

# **3. Model**
## **a. DenseNet**

In [10]:
# DenseNet
class DenseLayer(nn.Module):
    def __init__(self, input_features, growth_rate, bn_size):
        super(DenseLayer, self).__init__()
        self.bn1 = nn.BatchNorm2d(input_features)
        self.conv1 = nn.Conv2d(input_features, bn_size * growth_rate, kernel_size=1, stride=1, bias=False)
        self.bn2 = nn.BatchNorm2d(bn_size * growth_rate)
        self.conv2 = nn.Conv2d(bn_size * growth_rate, growth_rate, kernel_size=3, stride=1, padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        out = torch.cat([x, out], 1)    # concatenate all previous feature maps and the output feature maps of current layer
        return out

class DenseBlock(nn.Module):
    def __init__(self, num_layers, input_features, bn_size, growth_rate):
        super(DenseBlock, self).__init__()
        layers = []
        for i in range(num_layers):
            layers.append(DenseLayer(input_features + i * growth_rate, growth_rate, bn_size))   # #feature_in = #input_features + i*growth_rate, #feature_out = growth_rate
        self.layers = nn.Sequential(*layers)    # * : unpacking the list "layers" and combine them with nn.Sequential

    def forward(self, x):
        return self.layers(x)

class TransitionLayer(nn.Module):
    def __init__(self, input_features, output_features):
        super(TransitionLayer, self).__init__()
        self.bn = nn.BatchNorm2d(input_features)
        self.conv = nn.Conv2d(input_features, output_features, kernel_size=1, stride=1, bias=False)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        return self.pool(self.conv(F.relu(self.bn(x))))

class DenseNet(nn.Module):
    def __init__(
            self,
            growth_rate=12,
            block_config=(6, 12, 24, 16),
            compression=0.5,
            num_init_features=64,
            bn_size=4,
            num_classes=2
        ):
        super(DenseNet, self).__init__()
        # Initial convolution
        self.features = nn.Sequential(
            nn.Conv2d(3, num_init_features, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(num_init_features),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )   # #feature_map: 3 -> 64

        # Dense Blocks
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = DenseBlock(num_layers, num_features, bn_size, growth_rate)  # #layer = block_config[i], #features = #init_features + i* num_layers * growth_rate
            self.features.add_module(f"denseblock{i+1}", block)
            num_features = num_features + num_layers * growth_rate              # update num_features for next block
            # add transition layer if it's not the last block
            if i < len(block_config) - 1:
                trans = TransitionLayer(num_features, int(num_features * compression))
                self.features.add_module(f"transition{i+1}", trans)
                num_features = int(num_features * compression)

        # Final batch norm
        self.features.add_module('norm5', nn.BatchNorm2d(num_features))

        # Classifier
        self.classifier = nn.Linear(num_features, num_classes)

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.avg_pool2d(out, kernel_size=7, stride=1).view(features.size(0), -1)
        out = self.classifier(out)
        return out

## **b. FaceResNet**

In [11]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=2):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

def ResNet18(num_classes=2):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes)

# **c. Ensemble**

In [12]:
class MultiLayerNN(nn.Module):
    def __init__(self, input_size, nLayers, nNeurons, output_size):
        super(MultiLayerNN, self).__init__()
        self.L_in = nn.Linear(input_size, nNeurons)
        self.layers = nn.ModuleList()
        for _ in range(nLayers):
            self.layers.append(nn.Linear(nNeurons, nNeurons))
        self.L_out = nn.Linear(nNeurons, output_size)
    def forward(self, x):
        x = torch.flatten(x, 1) #Data x 28 x 28 -> #Data x 784
        x = torch.relu(self.L_in(x))
        for layer in self.layers:
            x = torch.relu(layer(x))
        x = self.L_out(x)
        return x

# **4. Train**

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [18]:
# Training
def TrainModel(model, trainloader, testloader, epochs=10, lr=1e-3, save_model = True, model_name = "DenseNet"):
    model_name = model_name + '.pth'
    model.to(device)  # Move the model to the GPU
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)
    criterion = nn.CrossEntropyLoss()

    model.train()  # training mode
    lossRec = [[],[]]
    AccRec = [[],[]]
    best_test_acc = 60

    for ep in range(epochs):
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move the data to the GPU
            optimizer.zero_grad()  # Resets the gradients of all optimized `torch.Tensor`s.
            outputs = model(inputs)  # forward propagation
            loss = criterion(outputs, labels)  # calculate loss
            loss.backward()  # backward propagation
            optimizer.step()  # update weights

        trainloss, trainacc = EvalModel(model, trainloader)
        testloss, testacc = EvalModel(model, testloader)
        lossRec[0].append(trainloss)
        lossRec[1].append(testloss)
        AccRec[0].append(trainacc)
        AccRec[1].append(testacc)
        print(f"Epoch {ep+1} >>> Train Loss: {trainloss} Accuracy: {trainacc}% <<< Test Loss: {testloss} Accuracy: {testacc}%")
        scheduler.step()  # Adjust lr with scheduler

        if save_model and testacc > best_test_acc:
            best_test_acc = testacc
            torch.save(model.state_dict(), fp+'/'+model_name)
            print("Model saved!")

    return lossRec, AccRec

# Testing
def EvalModel(model, dataloader):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()  # evaluation mode
    criterion = nn.CrossEntropyLoss()
    _loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move the data to the GPU
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)  # get predicted result
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    loss_avg = _loss / len(dataloader)
    accuracy = 100 * correct / total
    return loss_avg, accuracy

In [15]:
densenet1 = DenseNet(
            growth_rate=12,
            block_config=(6, 12, 24, 16),
            compression=0.5,
            num_init_features=64,
            bn_size=4,
            num_classes=2
        )
resnet1 = ResNet18()
resnet2 = ResNet18()
multilayer_nn = MultiLayerNN(5,4,20,2)

In [ ]:
# DenseNet
transform1 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.2,0.2,0.2]),
])
trainset_ds = ImageFolder(fp_ds+'/train', transform=transform1)
testset_ds = ImageFolder(fp_ds+'/test', transform=transform1)
trainloader_ds = DataLoader(trainset_ds, batch_size=32, shuffle=True)
testloader_ds = DataLoader(testset_ds, batch_size=32, shuffle=False)
DenseNet_loss, DenseNet_acc = TrainModel(densenet1, trainloader_ds, testloader_ds, epochs=20, lr=1e-3, save_model = True, model_name = "DenseNet")

In [20]:
# FaceResNet
transform2 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.2,0.2,0.2]),
])
trainset_face2 = ImageFolder(fp_face+'2/train', transform=transform2)
testset_face2 = ImageFolder(fp_face+'2/test', transform=transform2)
trainloader_face2 = DataLoader(trainset_face2, batch_size=32, shuffle=True)
testloader_face2 = DataLoader(testset_face2, batch_size=32, shuffle=False)
ResNet1_loss, ResNet1_acc = TrainModel(resnet1, trainloader_face2, testloader_face2, epochs=20, lr=1e-3, save_model = True, model_name = "FaceResNet")

Epoch 1 >>> Train Loss: 0.6320363548066881 Accuracy: 61.91304347826087% <<< Test Loss: 0.6947482004761696 Accuracy: 59.66386554621849%
Epoch 2 >>> Train Loss: 0.6708187576797273 Accuracy: 63.65217391304348% <<< Test Loss: 0.794597253203392 Accuracy: 57.983193277310924%
Epoch 3 >>> Train Loss: 0.6127268473307291 Accuracy: 67.30434782608695% <<< Test Loss: 0.6602290868759155 Accuracy: 63.865546218487395%
Model saved!
Epoch 4 >>> Train Loss: 0.5731456014845107 Accuracy: 71.82608695652173% <<< Test Loss: 0.6260439530014992 Accuracy: 70.58823529411765%
Model saved!
Epoch 5 >>> Train Loss: 0.6536429458194308 Accuracy: 58.95652173913044% <<< Test Loss: 0.682550385594368 Accuracy: 52.94117647058823%
Epoch 6 >>> Train Loss: 0.5555032210217582 Accuracy: 73.04347826086956% <<< Test Loss: 0.5848346874117851 Accuracy: 68.90756302521008%
Epoch 7 >>> Train Loss: 0.5805496623118719 Accuracy: 68.17391304347827% <<< Test Loss: 0.6842668578028679 Accuracy: 59.66386554621849%
Epoch 8 >>> Train Loss: 0.522

In [23]:
# FaceResNet
transform3 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.2,0.2,0.2]),
])
trainset_face3 = ImageFolder(fp_face+'3/train', transform=transform2)
testset_face3 = ImageFolder(fp_face+'3/test', transform=transform2)
trainloader_face3 = DataLoader(trainset_face3, batch_size=32, shuffle=True)
testloader_face3 = DataLoader(testset_face3, batch_size=32, shuffle=False)
ResNet2_loss, ResNet2_acc = TrainModel(resnet2, trainloader_face3, testloader_face3, epochs=20, lr=1e-3, save_model = True, model_name = "FaceResNetLBP")

Epoch 1 >>> Train Loss: 0.517496281199985 Accuracy: 73.04347826086956% <<< Test Loss: 0.5480726286768913 Accuracy: 76.47058823529412%
Model saved!
Epoch 2 >>> Train Loss: 0.5363896158006456 Accuracy: 74.08695652173913% <<< Test Loss: 0.5215811878442764 Accuracy: 73.94957983193277%
Epoch 3 >>> Train Loss: 0.5439895259009467 Accuracy: 69.21739130434783% <<< Test Loss: 0.5605466216802597 Accuracy: 62.18487394957983%
Epoch 4 >>> Train Loss: 0.5344344725211462 Accuracy: 71.1304347826087% <<< Test Loss: 0.5518599078059196 Accuracy: 70.58823529411765%
Epoch 5 >>> Train Loss: 0.5541662590371238 Accuracy: 72.34782608695652% <<< Test Loss: 0.5861912369728088 Accuracy: 62.18487394957983%
Epoch 6 >>> Train Loss: 0.5173406104246775 Accuracy: 77.3913043478261% <<< Test Loss: 0.5279541611671448 Accuracy: 78.15126050420169%
Model saved!
Epoch 7 >>> Train Loss: 0.4519716335667504 Accuracy: 79.30434782608695% <<< Test Loss: 0.5056336000561714 Accuracy: 73.10924369747899%
Epoch 8 >>> Train Loss: 0.440252

## **Ensemble**

In [51]:
def ProbListDataLoader(fp, densenet_model, resnet_model, batch_size=32):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    mtcnn = MTCNN(keep_all=True, device=device)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.2, 0.2, 0.2])
    ])

    labeldict = {
        "adults": 0,
        "children": 1
    }

    features_list = []
    labels_list = []

    for subdir in os.listdir(fp):
        subdir_path = os.path.join(fp, subdir)
        if os.path.isdir(subdir_path):  # Ensure it is a directory
            label = subdir  # Use the subdirectory name as the label
            for filename in os.listdir(subdir_path):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_path = os.path.join(subdir_path, filename)
                    try:
                        image = Image.open(image_path).convert('RGB')
                        image_tensor = transform(image).unsqueeze(0).to(device)

                        # Predict with DenseNet
                        with torch.no_grad():
                            densenet_model.eval()
                            densenet_output = densenet_model(image_tensor)
                            densenet_prob = torch.nn.functional.softmax(densenet_output, dim=1).squeeze().tolist()[0:2]  # Take the first two probabilities

                        # Process image for face detection and LBP feature extraction
                        image_rgb = np.array(image)
                        boxes, probs = mtcnn.detect(image)

                        face_detected = 0
                        if boxes is not None and probs is not None and len(probs) > 0:
                            max_prob_index = np.argmax(probs)
                            box = boxes[max_prob_index]
                            x_min, y_min, x_max, y_max = map(int, box)
                            x_min = max(x_min, 0)
                            y_min = max(y_min, 0)
                            x_max = min(x_max, image_rgb.shape[1])
                            y_max = min(y_max, image_rgb.shape[0])

                            face_image = image_rgb[y_min:y_max, x_min:x_max]
                            if face_image.size > 0:
                                gray_face = cv2.cvtColor(face_image, cv2.COLOR_RGB2GRAY)
                                equalized_face = cv2.equalizeHist(gray_face)
                                lbp = local_binary_pattern(equalized_face, 8, 1, method='uniform')
                                lbp_normalized = cv2.normalize(lbp, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
                                lbp_rgb = np.stack((lbp_normalized,) * 3, axis=-1)  # Convert grayscale to RGB
                                lbp_image = Image.fromarray(lbp_rgb)
                                lbp_tensor = transform(lbp_image).unsqueeze(0).to(device)

                                with torch.no_grad():
                                    resnet_model.eval()
                                    resnet_output = resnet_model(lbp_tensor)
                                    resnet_prob = torch.nn.functional.softmax(resnet_output, dim=1).squeeze().tolist()[0:2]  # Take the first two probabilities

                                features = densenet_prob + resnet_prob + [1]
                                face_detected = 1
                            else:
                                features = densenet_prob + [0.5, 0.5] + [0]
                        else:
                            features = densenet_prob + [0.5, 0.5] + [0]

                        features_list.append(features)
                        labels_list.append(labeldict[label])  # Append the subdirectory name as the label
                    except Exception as e:
                        print(f"Error processing {filename}: {str(e)}")

    # Convert lists to tensors
    features_tensor = torch.tensor(features_list, dtype=torch.float32)
    labels_tensor = torch.tensor(labels_list)  # This may need to be converted to numerical labels depending on your application

    # Create TensorDataset and DataLoader
    dataset = TensorDataset(features_tensor, labels_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    return dataloader

In [53]:
densenet_model = DenseNet(
            growth_rate=12,
            block_config=(6, 12, 24, 16),
            compression=0.5,
            num_init_features=64,
            bn_size=4,
            num_classes=2
        )
resnet_model = ResNet18()
densenet_model_path = os.path.join(fp, 'DenseNet.pth')
resnet_model_path = os.path.join(fp, 'FaceResNetLBP.pth')
densenet_state_dict = torch.load(densenet_model_path)
resnet_state_dict = torch.load(resnet_model_path)
densenet_model.load_state_dict(densenet_state_dict)
resnet_model.load_state_dict(resnet_state_dict)

trainloader_nn = ProbListDataLoader(fp_ds+'/train', densenet_model, resnet_model)
testloader_nn = ProbListDataLoader(fp_ds+'/test', densenet_model, resnet_model)

In [56]:
# train multilayer with p_list
multilayer_nn = MultiLayerNN(5,4,20,2)
# p list to dataloader
NN_Loss, NN_Acc = TrainModel(multilayer_nn, trainloader_nn, testloader_nn, epochs=50, lr=1e-3, save_model = True, model_name = "MultiLayerNN")

Epoch 1 >>> Train Loss: 0.6920076840453677 Accuracy: 50.0% <<< Test Loss: 0.6924751251935959 Accuracy: 50.0%
Epoch 2 >>> Train Loss: 0.6898316641648611 Accuracy: 50.0% <<< Test Loss: 0.6906544417142868 Accuracy: 50.0%
Epoch 3 >>> Train Loss: 0.6844618684715695 Accuracy: 50.0% <<< Test Loss: 0.6853564381599426 Accuracy: 50.0%
Epoch 4 >>> Train Loss: 0.6696820590231154 Accuracy: 69.10714285714286% <<< Test Loss: 0.6735213696956635 Accuracy: 66.66666666666667%
Model saved!
Epoch 5 >>> Train Loss: 0.6305071115493774 Accuracy: 78.75% <<< Test Loss: 0.6407751590013504 Accuracy: 75.0%
Model saved!
Epoch 6 >>> Train Loss: 0.5460724648502138 Accuracy: 83.21428571428571% <<< Test Loss: 0.5708601474761963 Accuracy: 80.0%
Model saved!
Epoch 7 >>> Train Loss: 0.43930487003591323 Accuracy: 82.85714285714286% <<< Test Loss: 0.49282679706811905 Accuracy: 80.83333333333333%
Model saved!
Epoch 8 >>> Train Loss: 0.3769810199737549 Accuracy: 83.39285714285714% <<< Test Loss: 0.4661629945039749 Accuracy: 7

# **5.Test**

# **6. Demo**

In [ ]:
# =============================
# Predict
# =============================

# =============================
# Model info
# =============================

# PreProcessing

# DenseNet

# ResNet

# SUM
